# Kubernetes

The following note is for personal purposes for understanding the architecture of [kubernetes](https://kubernetes.io/) with the resources from [edx](https://learning.edx.org/course/course-v1:LinuxFoundationX+LFS158x+1T2022/block-v1:LinuxFoundationX+LFS158x+1T2022+type@sequential+block@c77f2d1b11234afb9886680612041ab2/block-v1:LinuxFoundationX+LFS158x+1T2022+type@vertical+block@6893f0526bfe4a8c81ae62633aa6f06b).

Legacy softwares that use assembly codes are highly vulnerable to downtime when it comes to updating features and developing new features. Such applications are **Monolithic** and very inefficient, as opposed to modern architecture that has functions broken down into microservices or **Pebbles**. 

**Index**
- [Container](#container)
- [Container Orchestration](#container-orchestration)
- [Kubernetes](#kubernetes-summary)
- [Kubernetes Architecture](#kubernetes-architecture)
- [Control Plane Node Component](#control-plane-node-component)
  - [API Server](#api-server)
  - [Scheduler](#scheduler)
  - [Controller Manager](#controller-managers)
  - [Kube-Controller-Manager](#kube-controller-manager)
  - [Cloud-Controller-Manager](#cloud-controller-manager)
  - [key-value data store](#key-value-data-store)
- [Worker Node](#worker-node)
  - [Container Runtime](#container-runtime)
  - [kubelet](#kubelet)
  - [Kube-Proxy](#kube-proxy)
  - [addons](#addons)



### Container

Before learning about kubernetes, it is important to understand the key concepts of what a [Container](https://learning.edx.org/course/course-v1:LinuxFoundationX+LFS158x+1T2022/block-v1:LinuxFoundationX+LFS158x+1T2022+type@sequential+block@c77f2d1b11234afb9886680612041ab2/block-v1:LinuxFoundationX+LFS158x+1T2022+type@vertical+block@453babddfd324944b4f6623bd18f3536) is.

**Containers** are an *application-centric method to deliver high-performing, scalable applications on any infrastructure of choice*. Containers are best suited to deliver microservices by providing portable isolated virtual environments for applications to run without interference from other running applications. 

![pic](./pics/containers.png)

__Microservices__ are lightweight applications written within various modern programming languages, with specific dependencies, libraries and envrionmental requirements.

***Container*** does not run the apps directly, but the **Container Images**. A **Container Image** bundles the application along with its runtime, libraries and dependencies. 


### Container Orchestration

It would be unwise to run all the microservices in a single **Container**, because it would be counterintuitive to have the application divided into microservices in the first place. Therefore, when an application is broken down into microservices, it is deployed in multiple **Containers** allowing each microservices to be managed separately. ***Container Orchestration*** are tools which group systems together to form clusters where container's deployment and management is automated at scale while meeting the requirements for Quality Assurance(QA) and Production environments.

Most ***Container Orchestrators*** can:

- Group hosts together while creating a cluster
- Schedule containers to run on hosts in the cluster based on resources availability.
- Enable containers in a cluster to communicate with each other regardless of the host they are deployed to in the cluster.
- Bind containers and storage resources
- Group sets of similar containers and bind them to load-balancing constructs to simplify access to containerized applications by creating an interface, a level of abstraction between the containers and the client
- Manage and optimize resource usage
- Allow for implementation of policies to secure access to applications running inside containers.


### Kubernetes Summary

According to the [Kubernetes](https://kubernetes.io/) website,

**_'Kubernetes is an open-source system for automating deployment, scaling, and management of containerized applications'_**


- Automatic bin packing
  
  Kubernetes automatically schedules containers based on resource needs and constraints, to maximize utilization without sacrificing availability.
- Designed for extensibility
  
  A Kubernetes cluster can be extended with new custom features without modifying the upstream source code.
- Self-healing

  Kubernetes automatically replaces and reschedules containers from failed nodes. It terminates and then restarts containers that become unresponsive to health checks, based on existing rules/policy. It also prevents traffic from being routed to unresponsive containers.
- Horizontal scaling

  With Kubernetes applications are scaled manually or automatically based on CPU or custom metrics utilization.
- Service discovery and load balancing
  Containers receive IP addresses from Kubernetes, while it assigns a single Domain Name System (DNS) name to a set of containers to aid in load-balancing requests across the containers of the set.

- Automated rollouts and rollbacks

  Kubernetes seamlessly rolls out and rolls back application updates and configuration changes, constantly monitoring the application's health to prevent any downtime.
- Secret and configuration management
  
  Kubernetes manages sensitive data and configuration details for an application separately from the container image, in order to avoid a re-build of the respective image. Secrets consist of sensitive/confidential information passed to the application without revealing the sensitive content to the stack configuration, like on GitHub.
- Storage orchestration
  
  Kubernetes automatically mounts software-defined storage (SDS) solutions to containers from local storage, external cloud providers, distributed storage, or network storage systems.
- Batch execution

  Kubernetes supports batch execution, long-running jobs, and replaces failed containers.
- IPv4/IPv6 dual-stack
  
  Kubernetes supports both IPv4 and IPv6 addresses.

### Kubernetes Architecture

At a very high level, kuberneetes is a cluster of compute systems categorized by their distinct roles.

<img src="./pics/Components_of_Kubernetes_Architecture.png" alt="Kubernetes Architecture" title="Kubernetes Architecture"/> 


### Control Plane Node Component


**Control Plane Node** Provides a running environment for the **control plane agents** responsible for managing the state of a kubernetes cluster, it can be seen as the brain behind all operations inside the cluster. Therefore, as important the role of **Control Plane Node** is, it is important that it remains running at all times to prevent *downtime*, *service disruptions* which will ultimately lead to business losses. 


##### API Server

All the administrative tasks are coordinated by the **kube-apiserver**. The API Server intercepts RESTful calls from users, administrators, developers, operators and external agents, then validates and processes them

##### Scheduler

The role of **Kube-Scheduler** is to assign new workload objects, such as pods encapsulating containers, to nodes(Typically woker nodes). The scheduler obtains from the key-value store, via the API Server, resource usage data for each worker node in the cluster.

##### Controller Managers

**Controller Managers** are components of the control plane node running controllers or operator processes to regulate the state of the kubernetes cluster. 

##### Kube-Controller-manager

**Kube-Controller-Manager** runs controllers or operators responsible to act when nodes become unavailable, to ensure container pod counts are as expected, to create endpoints, service accounts, and API access tokens.

##### Cloud-Controller-Manager

It runs controllers or operators responsible to interact with the underlying infrastructure of a cloud provider when nodes become unavailable, to manage storage volumes when provided by a cloud service, and to manage load balancing and routing.

##### key-value data store

[etcd](https://etcd.io/) is an open source project under the CNCF, and it is strongly consistent, distributed *key-value data store* used to persist a kubernetes cluster's state.

### Worker Node

A *Worker node* provides a running environment for client applications. Though containerized microservices, these applications are encapsulated in Pods, controlled by the cluster control plane agents running on the control plane node. Pods are scheduled on worker nodes, where they find required compute, memory and storage resources to run, and networking to talk to each other and the outside world. A Pod is the smallest scheduling work unit in Kubernetes. It is a logical collection of one or more containers scheduled together, and the collection can be started, stopped, or rescheduled as a single unit of work. 

##### Container Runtime

Kubernetes does not have direct handle over containers, it is the role of __Container Runtime__ to manage the containers lifecycle. Kubernetes supports several container runtimes:

- [CRI-O](https://cri-o.io/)
  - A lightweight container runtime for kubernetes, supporting [quay.io](https://quay.io/) and [Docker Hub](https://hub.docker.com/) image registries
- [containerd](https://containerd.io/)
  - A simple, robust and portable container runtime
- [Docker](https://www.docker.com/)
  - A popular and complex container platform which uses __containerd__ as a container runtime
- [Mirantis Container Runtime](https://www.mirantis.com/software/container-runtime/)
  - Formerly known as the __Docker Enterprise Edition__

##### Kubelet

__Kubelet__ is an agent running on each node, control plane and workers. It communicates with the control plane. It also receives pod definitions from API server, and interacts with the container runtime on the node to run containers associated with the pod. 

__Kubelet__ connects to container runtimes through a plugin based interface the [container runtime interface (CRI)](https://github.com/kubernetes/community/blob/master/contributors/devel/sig-node/container-runtime-interface.md). 

![img](./pics/CRI.png)

As shown above, the kubelet acting as [gRPC](https://en.wikipedia.org/wiki/GRPC) client connects to the CRI [shim](https://en.wikipedia.org/wiki/Shim_(computing)) acting as grpc server to perform container and image operations. The CRI implements two services: __ImageService__ and __RuntimeService__.

__ImageService__ is responsible for all the image-related operations

__RuntimeService__ is responsible for all the pod and container-related operations.

##### Kube-Proxy



##### Addons